In [55]:
#airflow hb ecg temp gsr pid
airflow=3
heart_beat=103
ecgdata=60
temp=96
gsrdata=50
pid=12345

In [56]:
initialData={'airflow':airflow,
             'heart_beat':heart_beat,
             'ecgdata':ecgdata,
             'temp':temp,
             'gsrdata':gsrdata,
             'pid':pid}

#**Homomorphic**

In [57]:
import time
import psutil

In [58]:
!pip install phe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
from phe import paillier

In [60]:
public_key, private_key = paillier.generate_paillier_keypair()
keys=['airflow','heart_beat','ecgdata','temp','gsrdata']
def homomorphicEncryption(public_key,initialData):
  encryptedData = {}
  encryptedDataObj={}
  for i in keys:
    encryptedDataObj[i]=public_key.encrypt(initialData[i])
    encryptedData[i]=public_key.encrypt(initialData[i]).ciphertext()
  encryptedData['pid']=initialData['pid']
  encryptedDataObj['pid']=initialData['pid']
  print(encryptedData)
  return encryptedDataObj

In [61]:
begin=time.time()
encryptedDataObj=homomorphicEncryption(public_key,initialData)
end=time.time()
print("Time Taken for Homomorphic Encryption: ",end-begin)
print('The CPU usage for Homomorphic Encryption: ', psutil.cpu_percent(end-begin))

{'airflow': 8890328731975552448172009033990318876451220593112329772848272258191427989878255981942512904294703159556661168618146359970783207386343775531095474956108534571495673708141039139750656123301287269493751595454996508053464010526604158837824010639403354186480854807457148159244615118578875933388588500038430858600875061292642526961380975911424458828030972664346870338438152547762599422006538812133522291340319646930704611177800293482043440065423916223995765753187657725145673208012628141192136960985462848889696489425472600293183583623659334401236864528653122108122005993466148008486317664868894854800576395909061077086807018779591359463526197906606260885220570677102905433444258525092638486319748726761262761131838258392086325123755540460536613270365073337665053538187414408808745253915183426166685397238089689048999997145204911262162402094922403685868697863804985704716444154883996840389489520178955673862875870721089911658857768968006053789184766542033122049570358499049220595753067415913374

In [62]:
def homomorphicDecryption(private_key,encryptedDataObj):
  decryptedData = {}
  for i in keys:
    decryptedData[i]=private_key.decrypt(encryptedDataObj[i])
  decryptedData['pid']=encryptedDataObj['pid']
  print(decryptedData)
  return decryptedData

In [63]:
begin=time.time()
decryptedData=homomorphicDecryption(private_key,encryptedDataObj)
end=time.time()
print("Time Taken for Homomorphic Decryption: ",end-begin)
print('The CPU usage for Homomorphic Decryption: ', psutil.cpu_percent(end-begin))

{'airflow': 3, 'heart_beat': 103, 'ecgdata': 60, 'temp': 96, 'gsrdata': 50, 'pid': 12345}
Time Taken for Homomorphic Decryption:  0.5504150390625
The CPU usage for Homomorphic Decryption:  2.8


#**Acorn**

In [64]:
!pip install pycryptodomex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad, unpad
import base64
key = '1234567891234567'
keys=['airflow','heart_beat','ecgdata','temp','gsrdata']

In [66]:
def acornEncryption(key,initialData):
  key_bytes = key.encode('utf-8')
  cipher = AES.new(key_bytes, AES.MODE_ECB)
  encryptedData = {}
  for i in keys:
    plaintext_bytes = str(initialData[i]).encode('utf-8')
    padded_plaintext = pad(plaintext_bytes, AES.block_size)
    ciphertext_bytes = cipher.encrypt(padded_plaintext)
    ciphertext_string = base64.b64encode(ciphertext_bytes).decode('utf-8')
    encryptedData[i]=ciphertext_string
  encryptedData['pid']=initialData['pid']
  print(encryptedData)
  return encryptedData

In [67]:
begin=time.time()
encryptedData=acornEncryption(key,initialData)
end=time.time()
print("Time Taken for Acorn Encryption: ",end-begin)
print('The CPU usage for Acorn Encryption: ', psutil.cpu_percent(end-begin))

{'airflow': 'DY/MIIKVXh3qD8oh4B19Ow==', 'heart_beat': 'iZ+OJR0rh3sEvMWotd8P4Q==', 'ecgdata': 'bhH9M8qFkZxb85COLgk0Pg==', 'temp': 'O4bpAkLEl86Aw2QV22xepA==', 'gsrdata': 'UzdMtnZaFOQGv8Q8Cglslw==', 'pid': 12345}
Time Taken for Acorn Encryption:  0.0009090900421142578
The CPU usage for Acorn Encryption:  100.0


In [68]:
def acornDecryption(key,encryptedData):
  key_bytes = key.encode('utf-8')
  decryptedData = {}
  for i in keys:
    ciphertext_bytes = base64.b64decode(encryptedData[i].encode('utf-8'))
    decipher = AES.new(key_bytes, AES.MODE_ECB)
    decrypted_plaintext_bytes = unpad(decipher.decrypt(ciphertext_bytes), AES.block_size)
    decrypted_plaintext_str = decrypted_plaintext_bytes.decode('utf-8')
    decryptedData[i]=decrypted_plaintext_str
  decryptedData['pid']=encryptedData['pid']
  print(decryptedData)
  return decryptedData

In [69]:
begin=time.time()
decryptedData=acornDecryption(key,encryptedData)
end=time.time()
print("Time Taken for Acorn Decryption: ",end-begin)
print('The CPU usage for Acorn Decryption: ', psutil.cpu_percent(end-begin))

{'airflow': '3', 'heart_beat': '103', 'ecgdata': '60', 'temp': '96', 'gsrdata': '50', 'pid': 12345}
Time Taken for Acorn Decryption:  0.002666950225830078
The CPU usage for Acorn Decryption:  0.0
